# Titanic ML Exploration - Scikitlearn
This is my exploratory attempt to develop a machine learning algorithm using the training dataset and the sklearn package. Based on the Kaggle tutorial available at https://github.com/savarin/pyconuk-introtutorial/blob/master/notebooks/Section%201-0%20-%20First%20Cut.ipynb

I put all 'data' in the data directory

## Data Cleaning

In [91]:
#load package dependencies
import numpy as np
import pandas as pd
import sklearn

In [92]:
train = pd.read_csv('~/Documents/titanic/data/train.csv')

In [93]:
#take a look at the data
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [94]:
#drop unnecessary vars
train = train.drop(['Name', 'Cabin', 'Ticket'], axis=1)

In [95]:
#fill in na values for age
age_mean = train['Age'].mean()
train['Age'] = train['Age'].fillna(age_mean)

In [96]:
#sklearn only takes numerical input, so we need to convert categorical vars (Sex, Embarked) to integers
train['GENDER'] = train['Sex'].map( {'male' : 1, 'female' : 0} ).astype('int')
train['Embarked'] = train['Embarked'].fillna('none') #get rid of those two pesky NA's
train['PORT'] = train['Embarked'].map( {'C' : 1, 'S' : 2, 'Q' : 3, 'none' : 4} ).astype('int')

In [97]:
#drop recoded vars
train = train.drop(['Sex', 'Embarked'], axis=1)

In [98]:
#view columns columns
cols = train.columns.tolist()
print(cols)

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'GENDER', 'PORT']


In [99]:
#reorder columns so that the column we're trying to predict is in the 0 position
cols = [cols[1]] + cols[0:1] + cols[2:]
train = train[cols]

In [100]:
#check cleaned dataset
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived       891 non-null int64
PassengerId    891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
GENDER         891 non-null int64
PORT           891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.7 KB


,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,GENDER,PORT
0,0,1,3,22.0,1,0,7.2500,1,2
1,1,2,1,38.0,1,0,71.2833,0,1
2,1,3,3,26.0,0,0,7.9250,0,2
3,1,4,1,35.0,1,0,53.1000,0,2
4,0,5,3,35.0,0,0,8.0500,1,2


In [101]:
#convert cleaned dataset to numpy array
train_array = train.values

## Using Sklearn to Train a Model

In [102]:
#use randomforest
from sklearn.ensemble import RandomForestClassifier

#create a random forest classifier with 100 trees
model = RandomForestClassifier(n_estimators = 100)

In [103]:
#Train the model using the cleaned dataset, omit PassengerID
model = model.fit(train_array[0:, 2:], train_array[0:, 0])

## Using Sklearn to make predictions

In [104]:
#load the test dataset
test = pd.read_csv('~/Documents/titanic/data/test.csv')

In [105]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [106]:
#clean test dataset (make it look the same as train data minus the column you're trying to predict)
#Need to generate filler values to get rid of the NaNs for age and fare
age_mean = test['Age'].mean()
test['Age'] = test['Age'].fillna(age_mean)

fare_median = test['Fare'].median()
test['Fare'] = test['Fare'].fillna(fare_median)

test['GENDER'] = test['Sex'].map( {'male' : 1, 'female' : 0} ).astype('int')
test['PORT'] = test['Embarked'].map( {'C' : 1, 'S' : 2, 'Q' : 3} ).astype('int')
test = test.drop(['Sex', 'Embarked', 'Name', 'Cabin', 'Ticket'], axis=1)

test_array = test.values

In [107]:
test.head()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
GENDER         418 non-null int64
PORT           418 non-null int64
dtypes: float64(2), int64(6)
memory usage: 26.2 KB


In [108]:
#Use the trained model to predict survival. Test dataset must have the same number of features as the model.
output = model.predict(test_array[:,1:])

## Prepare for Submission

In [109]:
result = test
result['Survived'] = output.astype(int)
result.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,GENDER,PORT,Survived
0,892,3,34.5,0,0,7.8292,1,3,0
1,893,3,47.0,1,0,7.0000,0,2,0
2,894,2,62.0,0,0,9.6875,1,3,0
3,895,3,27.0,0,0,8.6625,1,2,1
4,896,3,22.0,1,1,12.2875,0,2,0


In [112]:
#remove unnecessary vars for submission
clean_result = result.loc[:, ['PassengerId', 'Survived']]
clean_result.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0


In [113]:
#write to file
clean_result.to_csv('~/Downloads/titanic_submission.csv', index=False)